In [1]:
pip install haystack-ai 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers torch


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Import the necessary libraries

In [3]:
from dotenv import load_dotenv
import re
from pprint import pprint
from ast import literal_eval
from typing import List
import json
import pandas as pd

In [4]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import PromptBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.dataclasses import ChatMessage

2024-09-01 12:24:51.622485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 12:24:52.596324: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-01 12:24:55.017736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 12:24:55.017922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 12:24:55.874421: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [5]:
load_dotenv()

True

In [6]:
pip install --upgrade transformers

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install accelerate

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import gradio as gr
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret

# Loading the CLIP model and processor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Defining the feature descriptions
feature_descriptions = [
    "Source, Destination, and Date Selection",
    "Bus Selection",
    "Seat Selection",
    "Pick-up and Drop-off Point Selection"
]

# Initializing the text generator
generator = OpenAIGenerator(
    api_key=Secret.from_env_var("GROQ_API_KEY"),
    api_base_url="https://api.groq.com/openai/v1",
    model="llama-3.1-70b-versatile",
    generation_kwargs={"max_tokens": 2048}
)

# Converting images to embeddings using CLIP
def get_image_embedding(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# Generating the testing instructions based on the highest similarity feature
def generate_testing_instructions(context, images):
    results = []
    for image_path in images:
        img_features = get_image_embedding(image_path)
        similarities = []
        
        for desc in feature_descriptions:
            text_inputs = processor(text=desc, return_tensors="pt")
            text_features = clip_model.get_text_features(**text_inputs)
            similarity = torch.nn.functional.cosine_similarity(img_features, text_features)
            similarities.append((desc, similarity.item()))
        
        highest_similarity_desc = max(similarities, key=lambda x: x[1])[0]
        prompt = f"Generate detailed testing instructions for the following feature based on the provided screenshot: {highest_similarity_desc}. Include tests for visible functionalities and exclude unrelated features."

        try:
            result = generator.run(prompt=prompt)
            if "replies" in result:
                instruction = result["replies"][0].strip()
            else:
                instruction = "No result found"
        except Exception as e:
            instruction = f"Error generating instructions: {e}"
        
        results.append(f"Instructions for {image_path} (Feature: {highest_similarity_desc}):\n{instruction}\n---\n")
    
    return "\n".join(results)

# Setting up the Gradio interface
with gr.Blocks() as demo:
    with gr.Column():
        gr.Markdown("## Generate Testing Instructions")
        context = gr.Textbox(label="Optional Context", placeholder="Enter any context here...", lines=2)
        images = gr.Files(label="Upload Screenshots", file_count="multiple", type="filepath")
        submit_btn = gr.Button("Describe Testing Instructions")
        output = gr.Textbox(label="Generated Testing Instructions", lines=10)

    submit_btn.click(fn=generate_testing_instructions, inputs=[context, images], outputs=output)

demo.launch()


2024-09-01 13:14:30.144590: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 13:14:30.214251: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-01 13:14:30.262205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 13:14:30.262241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 13:14:30.263088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [10]:
pip install --upgrade pydantic


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 KB 377.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.3.0
    Uninstalling pydantic_core-2.3.0:
      Successfully uninstalled pydantic_core-2.3.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.18
    Uninstalling pydantic-1.10.18:
      Successfully uninstalled pydantic-1.10.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.8.3 requires matplotlib<3.9,>=3.2, but you have matplotlib 3.9.0 which is incompatible.
farm-haystack 1.26.3 requires pydantic<2, but you have pydantic 2.8.2 which is incompatible.
farm-haystack 1.26.3 re

In [3]:
pip install --upgrade gradio


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio-4.42.0-py3-none-any.whl (16.8 MB)
  Using cached gradio_client-1.3.0-py3-none-any.whl (318 kB)
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.6.1
    Uninstalling gradio_client-0.6.1:
      Successfully uninstalled gradio_client-0.6.1
  Attempting uninstall: gradio
    Found existing installation: gradio 3.50.0
    Uninstalling gradio-3.50.0:
      Successfully uninstalled gradio-3.50.0
Note: you may need to restart the kernel to use updated packages.
